In [2]:
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from pymatgen.ext.matproj import MPRester

C:\Users\dylan\AppData\Local\Temp\ipykernel_42896\2393419469.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [31]:
def findMinEigenvalue(c11, c12, c13, c21, c22, c23, c31, c32, c33):
    matrix = [[c11, c12, c13],[c21, c22, c23],[c31, c32, c33]]
    matrix = np.array(matrix).reshape((3,3))
    return min(eigs(matrix, 3, sigma=0)[0].real)
def classificationStiffness(stiff):
    if(stiff < 0):
        return 0
    else:
        return 1
def classificationThermo(heat, lowConvex):
    if(.2 < heat):
        return 0
    if(lowConvex + .2 < heat):
        return 1
    return 2

In [3]:
import requests
from bs4 import BeautifulSoup
import json
import time
import requests
import cchardet
from pymatgen.ext.matproj import MPRester

url = 'http://phonondb.mtl.kyoto-u.ac.jp/ph20180417/index.html'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
materialsGathered = []
groupings = []
index = 0
for link in soup.find_all('a'):
    if("mp" in link.get('href')):
        urls.append(link.get('href'))
        materialsGathered.append(link.get('href').rsplit('/', 1)[-1].rsplit('.html', 1)[0])
    index += 1

remaining = len(materialsGathered)
currentIndex = 0
while(remaining > 0):
    materialRequests = []
    for i in range(currentIndex, currentIndex + 1000):
        materialRequests.append(materialsGathered[i])
        remaining -= 1
    groupings.append(materialRequests)
    
url = 'http://phonondb.mtl.kyoto-u.ac.jp/ph20180417/index.html'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
namesOfMaterial = {"NotAMaterial":"mat"}

html = requests.get(url)
soup = BeautifulSoup(html.text, features="html.parser")
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

for element in chunks:
    if("/" in element):
        temp = element.split("/")
        namesOfMaterial[element.split("/")[0].split(" ")[2]] = temp[1][1:len(temp[1])-1]

In [4]:
elasticityClassifications = []
thermoClassifications = []
thermoIndexes = []
with MPRester("brO6w6AHZNfzGnTWuqOM") as m:
    dictionary = m.get_data("mp-21405")
    print(dictionary)
    numberOfAtoms = (dictionary[0]["energy"])/(dictionary[0]["energy_per_atom"])
    print((dictionary[0]["formation_energy_per_atom"]))
    print(dictionary[0]["e_above_hull"])
    print(classificationThermo(dictionary[0]["formation_energy_per_atom"], dictionary[0]["e_above_hull"]))

[{'energy': -14.38942371, 'energy_per_atom': -3.5973559275, 'volume': 119.394139826359, 'formation_energy_per_atom': -0.4735583236718752, 'nsites': 4, 'unit_cell_formula': {'In': 2.0, 'Se': 2.0}, 'pretty_formula': 'InSe', 'is_hubbard': False, 'elements': ['In', 'Se'], 'nelements': 2, 'e_above_hull': 0.036377234999999786, 'hubbards': {}, 'is_compatible': True, 'spacegroup': {'symprec': 0.1, 'source': 'spglib', 'symbol': 'C2/m', 'number': 12, 'point_group': '2/m', 'crystal_system': 'monoclinic', 'hall': '-C 2y'}, 'task_ids': ['mp-672921', 'mp-655946', 'mp-659122', 'mp-21405', 'mp-1421218', 'mp-1596039', 'mp-1700407'], 'band_gap': 1.0755000000000003, 'density': 5.3901462787971, 'icsd_id': None, 'icsd_ids': [32714, 71083], 'cif': "# generated using pymatgen\ndata_InSe\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   6.20754263\n_cell_length_b   6.20754263\n_cell_length_c   5.23058816\n_cell_angle_alpha   71.42219573\n_cell_angle_beta   71.42219573\n_cell_angle_gamma   39.00659361\

In [5]:
index = 0
entries = []
ignore = []
structs = []
materialsStiffnessAvailableFor = []
with MPRester("aU621ICtBbnsLKk9") as m:
    for material in materialsGathered:
        currentAttempts = 0
        success = False
        while(not success):
            try:
                dictionary = m.get_data(material)
                structure = m.get_structure_by_material_id(material)
                if(dictionary[0]["pretty_formula"] != namesOfMaterial[material[3:]]):
                    print("PhononDB: ", material, " MaterialsProject: ", dictionary[0]["pretty_formula"])
                    ignore.append(index)
                    index += 1
                    success = True
                else:
                    structs.append(structure)
                    try:
                        mat = dictionary[0]["elasticity"]["elastic_tensor_original"]
                        stiffnessClassification = classificationStiffness(findMinEigenvalue(mat[0][0], mat[0][1], mat[0][2], mat[1][0], mat[1][1], mat[1][2], mat[2][0], mat[2][1], mat[2][2]))
                        elasticityClassifications.append(stiffnessClassification)
                        materialsStiffnessAvailableFor.append(index)
                    except:
                        pass
                    try:
                        classify = classificationThermo(dictionary[0]["formation_energy_per_atom"], dictionary[0]["e_above_hull"])
                        thermoClassifications.append(classify)
                        thermoIndexes.append(classify)
                    except:
                        pass
                    index += 1
                    if((index - 1) % 100 == 0):
                        print("Progression: ", 100*index/len(materialsGathered))
                    success = True
            except:
                print("Waiting...for ", material)
                currentAttempts += 1
                if(currentAttempts == 3):
                    print("Ignoring...", material)
                    ignore.append(index)
                    index += 1
                    success = True

C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\scipy\sparse\linalg\_eigen\arpack\arpack.py:1265: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Progression:  0.009966115208291808
Progression:  1.0065776360374725
Progression:  2.0031891568666533
Progression:  2.999800677695834
Progression:  3.996412198525015
Progression:  4.993023719354196
Progression:  5.989635240183376
Progression:  6.986246761012557
Progression:  7.982858281841738
Progression:  8.979469802670918
Progression:  9.9760813235001
PhononDB:  mp-6182  MaterialsProject:  CsTb2Cu3Se5
PhononDB:  mp-6254  MaterialsProject:  Ba2TbNbO6
PhononDB:  mp-6265  MaterialsProject:  Ba2TbSbO6
Progression:  10.97269284432928
PhononDB:  mp-6449  MaterialsProject:  CdCu2SiS4
Progression:  11.969304365158461
PhononDB:  mp-6952  MaterialsProject:  RbYbF3
PhononDB:  mp-6964  MaterialsProject:  NaTbO2
PhononDB:  mp-7137  MaterialsProject:  LiTbO2
Progression:  12.965915885987641
Progression:  13.962527406816823
PhononDB:  mp-7895  MaterialsProject:  Rb2O2
Progression:  14.959138927646004
Progression:  15.955750448475184
PhononDB:  mp-8398  MaterialsProject:  CsYbF3
Progression:  16.9523

C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\ext\matproj.py:712: UserWarning: The calculation task mp-12992 is mapped to canonical mp-id mp-5986, so structure for mp-5986 returned. This is not an error, see documentation. If original task data for mp-12992 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn(


Waiting...for  mp-12992
Waiting...for  mp-12992
Waiting...for  mp-12992
Ignoring... mp-12992
Progression:  23.92864261510863
PhononDB:  mp-13872  MaterialsProject:  Cu8SiS6
Progression:  24.925254135937813
Progression:  25.921865656766993
Progression:  26.918477177596174
PhononDB:  mp-15793  MaterialsProject:  LiTbSe2
PhononDB:  mp-15895  MaterialsProject:  Cu2SiS3
Progression:  27.915088698425354
PhononDB:  mp-16594  MaterialsProject:  K3TbSi2O7
PhononDB:  mp-16773  MaterialsProject:  KTb(PO3)4
PhononDB:  mp-16927  MaterialsProject:  NaTbGeO4
Progression:  28.911700219254534
PhononDB:  mp-17153  MaterialsProject:  Ba(TbTe2)2
PhononDB:  mp-17486  MaterialsProject:  Ho3CuSiS7
Progression:  29.908311740083715
PhononDB:  mp-17513  MaterialsProject:  Ba4Tb2Cd3S10
PhononDB:  mp-17838  MaterialsProject:  KTb3F10
Progression:  30.904923260912895
PhononDB:  mp-18042  MaterialsProject:  SrTb2O4
PhononDB:  mp-18062  MaterialsProject:  Na10YbSn12
PhononDB:  mp-18084  MaterialsProject:  BaTb2ZnO5


C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\ext\matproj.py:712: UserWarning: The calculation task mp-42160 is mapped to canonical mp-id mp-41734, so structure for mp-41734 returned. This is not an error, see documentation. If original task data for mp-42160 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn(


...

Waiting...for  mp-646925
Waiting...for  mp-646925
Waiting...for  mp-646925
Ignoring... mp-646925


C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\ext\matproj.py:712: UserWarning: The calculation task mp-654956 is mapped to canonical mp-id mp-583071, so structure for mp-583071 returned. This is not an error, see documentation. If original task data for mp-654956 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn(


Waiting...for  mp-654956
Waiting...for  mp-654956
Waiting...for  mp-654956
Ignoring... mp-654956
PhononDB:  mp-655613  MaterialsProject:  Cs2Tb6Te7N2


C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\ext\matproj.py:712: UserWarning: The calculation task mp-656662 is mapped to canonical mp-id mp-8253, so structure for mp-8253 returned. This is not an error, see documentation. If original task data for mp-656662 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn(


Waiting...for  mp-656662
Waiting...for  mp-656662
Waiting...for  mp-656662
Ignoring... mp-656662
PhononDB:  mp-669304  MaterialsProject:  Cs3Tb2Br9
Progression:  81.73211082320111
PhononDB:  mp-673682  MaterialsProject:  Tb2YbS4
PhononDB:  mp-675767  MaterialsProject:  La2YbS4
Progression:  82.7287223440303
Progression:  83.72533386485948
PhononDB:  mp-696965  MaterialsProject:  Cs3TbF6
Progression:  84.72194538568866
Progression:  85.71855690651783
Progression:  86.71516842734702
Progression:  87.7117799481762
PhononDB:  mp-755044  MaterialsProject:  CaTb2O4
Progression:  88.70839146900538
Progression:  89.70500298983457
PhononDB:  mp-757197  MaterialsProject:  DyY3O6
Progression:  90.70161451066375
PhononDB:  mp-759928  MaterialsProject:  K3TbSi3(HO5)2
PhononDB:  mp-766292  MaterialsProject:  DyY7O12
Progression:  91.69822603149292
Progression:  92.6948375523221
Progression:  93.69144907315129
Progression:  94.68806059398047
Progression:  95.68467211480964
Progression:  96.6812836356

In [6]:
print(len(ignore))
print(len(elasticityClassifications))
print(len(thermoClassifications))
print(len(structs))

106
2095
9927
9928


In [7]:
import json
with open('outputClasses.json') as F:
    results = json.load(F)

In [8]:
forbiddenMPs = []
for index in ignore:
    forbiddenMPs.append(materialsGathered[index])

In [63]:
filelist = [file for file in os.listdir("phononDBImages") if file.endswith('.jpg')]
filelist = pd.read_csv('filelist.csv',header = None)
newFilelist = []
for entry in range(len(filelist)):
    thing = filelist.iloc[entry].values[0]
    newFilelist.append(thing)
print(len(filelist))
entries = pd.read_csv('entries.csv',header=None)
classification = {'mp-NOTAMATERIAL': True}
materialsProcessed = []
for fileIdx in range(len(filelist)):
    files = newFilelist[fileIdx]
    exampleImage = files[:len(files)-9]
    if(str(exampleImage) not in forbiddenMPs):
        classification[exampleImage] = results[exampleImage]

9927


In [53]:
materialStructures = []
classificationsForStructures = []
globalIndex = 0
for index in materialsStiffnessAvailableFor:
    materialStructures.append(materialsGathered[index])
    classificationsForStructures.append(elasticityClassifications[globalIndex])
    globalIndex += 1

In [54]:
print(len(elasticityClassifications))

2095


In [57]:
print(len(structs))

13896


In [71]:
print(classificationStiffness(findMinEigenvalue(12, -1, 7, -1, 12, 7, 7, 7, 5)))

0


In [23]:
db = connect('c2db.db')
length = 0
adapter = AseAtomsAdaptor()
arrayOfThings = []
for row in db.select():
    try:
        a = row['hform']
        a = row['ehull']
        varA = row['hform']
        varB = row['ehull']
        tempThis = classificationThermo(float(varA), float(varB))
        if(row['thermodynamic_stability_level'] + 1 != tempThis):
            print("HForm: ", varA, "EHull: ", varB, "Classification: ", (row['thermodynamic_stability_level']), "Ours: ", tempThis+1)
        arrayOfThings.append(tempThis)
    except:
        pass

HForm:  0.4247370934348935 EHull:  0.4247370934348935 Classification:  1 Ours:  1
HForm:  -1.064860946472334 EHull:  0.04936244159942782 Classification:  3 Ours:  3
HForm:  -0.7433614716609935 EHull:  0.35030501021094795 Classification:  2 Ours:  3
HForm:  -0.16756754153148634 EHull:  0.36280426507292063 Classification:  2 Ours:  3
HForm:  -0.4958010423209753 EHull:  0.03780502792618501 Classification:  3 Ours:  3
HForm:  -0.31377007784410793 EHull:  0.19105647442319462 Classification:  3 Ours:  3
HForm:  -1.0870792714193929 EHull:  -0.002629469935327533 Classification:  3 Ours:  3
HForm:  -1.8548282225228245 EHull:  -0.0670393496229571 Classification:  3 Ours:  3
HForm:  -2.9275122365498882 EHull:  -0.09373423588387153 Classification:  3 Ours:  3
HForm:  -1.270877318478873 EHull:  0.04125800310794148 Classification:  3 Ours:  3
HForm:  -2.954920238785681 EHull:  0.03838200004081349 Classification:  3 Ours:  3
HForm:  -0.15957109937497727 EHull:  -0.05336913346734673 Classification:  3

HForm:  -1.6219137250517093 EHull:  0.10671968024854683 Classification:  3 Ours:  3
HForm:  -1.585491476497457 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -1.6575703400756936 EHull:  0.25372522792029595 Classification:  2 Ours:  3
HForm:  -0.5897667907978125 EHull:  0.1675861279458417 Classification:  3 Ours:  3
HForm:  -2.4970091588280545 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -2.287543746602401 EHull:  0.17188120919671812 Classification:  3 Ours:  3
HForm:  -1.067907262108993 EHull:  0.023655733356264363 Classification:  3 Ours:  3
HForm:  -0.6557589834809207 EHull:  0.032819288303295946 Classification:  3 Ours:  3
HForm:  -1.0046006193331294 EHull:  0.11986881004493677 Classification:  3 Ours:  3
HForm:  -0.9422380525473131 EHull:  0.044237402440444384 Classification:  3 Ours:  3
HForm:  -0.3484992456298805 EHull:  0.18166747809276834 Classification:  3 Ours:  3
HForm:  -0.2250232222039017 EHull:  0.046478503479213396 Classification:  3 Ours:  3
HForm:  -1.2613834294651

HForm:  -0.1939446716216251 EHull:  0.15811748282783622 Classification:  3 Ours:  3
HForm:  -0.2655957770138473 EHull:  0.08635061283142154 Classification:  3 Ours:  3
HForm:  -0.21654895020602627 EHull:  0.5736332802173418 Classification:  2 Ours:  3
HForm:  -0.24536835143273272 EHull:  0.039974789168580216 Classification:  3 Ours:  3
HForm:  -2.3517584441126482 EHull:  0.6614486355201454 Classification:  2 Ours:  3
HForm:  -1.9547054482592747 EHull:  0.7270984950775201 Classification:  2 Ours:  3
HForm:  0.4445696015319278 EHull:  0.49784700362755974 Classification:  1 Ours:  1
HForm:  -0.34610404769762626 EHull:  0.07499368598771872 Classification:  3 Ours:  3
HForm:  0.24444814088836964 EHull:  0.5464214959494909 Classification:  1 Ours:  1
HForm:  -0.27873984058104195 EHull:  0.04968931321426773 Classification:  3 Ours:  3
HForm:  -0.056399607566345544 EHull:  0.44640914189441405 Classification:  2 Ours:  3
HForm:  -0.14502022754655197 EHull:  0.11891256071551137 Classification:  

HForm:  -0.48692850346437605 EHull:  0.14244282172880246 Classification:  3 Ours:  3
HForm:  -0.9828834523301554 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.7237933009560878 EHull:  0.01016825337213656 Classification:  3 Ours:  3
HForm:  -0.5808283891552412 EHull:  0.030703538658470864 Classification:  3 Ours:  3
HForm:  -0.08602573713757788 EHull:  0.0484299008284117 Classification:  3 Ours:  3
HForm:  -0.7716527329700025 EHull:  0.28748947380140466 Classification:  2 Ours:  3
HForm:  -0.06466602043242489 EHull:  0.17477346116472176 Classification:  3 Ours:  3
HForm:  -0.8056137861603873 EHull:  0.22054428314461372 Classification:  2 Ours:  3
HForm:  -0.17005446732539653 EHull:  0.28772804860856915 Classification:  2 Ours:  3
HForm:  -0.15417195913010176 EHull:  0.09054203815417519 Classification:  3 Ours:  3
HForm:  -0.5219203454229884 EHull:  0.09490989990580345 Classification:  3 Ours:  3
HForm:  0.20048196592494918 EHull:  0.36935280113616786 Classification:  1 Ours:  1
HFo

HForm:  -0.013443276197724785 EHull:  0.16824948562424427 Classification:  3 Ours:  3
HForm:  -2.095925664487821 EHull:  0.5147826523696417 Classification:  2 Ours:  3
HForm:  0.25950850916820895 EHull:  0.7373062032549851 Classification:  1 Ours:  1
HForm:  0.3144386122448258 EHull:  1.1879871863138654 Classification:  1 Ours:  1
HForm:  -0.10947177197261719 EHull:  0.026295396081462016 Classification:  3 Ours:  3
HForm:  -0.9781521855097312 EHull:  0.8002892260961586 Classification:  2 Ours:  3
HForm:  -0.16318965612867467 EHull:  0.31589011694087843 Classification:  2 Ours:  3
HForm:  -1.7280776995018445 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.24553077621492037 EHull:  0.3292018537849325 Classification:  2 Ours:  3
HForm:  -0.8188027845699836 EHull:  0.05033494653082715 Classification:  3 Ours:  3
HForm:  -0.942826229268794 EHull:  0.24152037785888159 Classification:  2 Ours:  3
HForm:  -2.3073893431821717 EHull:  0.16755116369806755 Classification:  3 Ours:  3
HForm:  -1

...

HForm:  -0.734675867809384 EHull:  0.014128517923407302 Classification:  3 Ours:  3
HForm:  -0.9703761607151429 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.6163521260424232 EHull:  0.01112302097780793 Classification:  3 Ours:  3
HForm:  -1.2152162729435687 EHull:  -0.01996854308309537 Classification:  3 Ours:  3
HForm:  -0.22463669703333977 EHull:  0.023288916713349067 Classification:  3 Ours:  3
HForm:  -0.26147983342390013 EHull:  0.029262844456182435 Classification:  3 Ours:  3
HForm:  -0.6294164889907868 EHull:  0.02721937668107377 Classification:  3 Ours:  3
HForm:  -0.4601930917824338 EHull:  -0.01792811494064611 Classification:  3 Ours:  3
HForm:  -0.3329356154600678 EHull:  0.04824877718066567 Classification:  3 Ours:  3
HForm:  -1.538292867272483 EHull:  0.03947072731617163 Classification:  3 Ours:  3
HForm:  -2.4664182256219394 EHull:  0.4413498730471441 Classification:  2 Ours:  3
HForm:  -1.6546665998230559 EHull:  -0.04640082450751071 Classification:  3 Ours:  3
HFo

HForm:  -1.9240008160234623 EHull:  0.01372836161300195 Classification:  3 Ours:  3
HForm:  -1.2135694421637673 EHull:  0.9641728299973262 Classification:  2 Ours:  3
HForm:  -2.9191924077997617 EHull:  8.648039961434151e-05 Classification:  3 Ours:  3
HForm:  -2.615726845128846 EHull:  0.20386724015180002 Classification:  2 Ours:  3
HForm:  -0.406776652338371 EHull:  0.1593734437888804 Classification:  3 Ours:  3
HForm:  -1.7251542170403495 EHull:  0.4108983376726809 Classification:  2 Ours:  3
HForm:  -1.1609347961582983 EHull:  0.08567805031255782 Classification:  3 Ours:  3
HForm:  -1.2330142950553957 EHull:  0.3126507020692828 Classification:  2 Ours:  3
HForm:  -1.5571277660964142 EHull:  0.3958499693287094 Classification:  2 Ours:  3
HForm:  -2.087332421856446 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -1.6654639387262498 EHull:  0.3485318364742118 Classification:  2 Ours:  3
HForm:  -0.0022462335588215865 EHull:  0.08064247147283128 Classification:  3 Ours:  3
HForm:  -0.2

HForm:  0.3668507166319932 EHull:  0.42205751230235933 Classification:  1 Ours:  1
HForm:  -0.25289348103617204 EHull:  0.16820425264917294 Classification:  3 Ours:  3
HForm:  -0.06841412109590195 EHull:  0.41045435396783303 Classification:  2 Ours:  3
HForm:  -0.37605803245437563 EHull:  0.050428525114218914 Classification:  3 Ours:  3
HForm:  -1.0451333405032344 EHull:  0.8551710919348929 Classification:  2 Ours:  3
HForm:  -0.7906661264515042 EHull:  0.6487061279619866 Classification:  2 Ours:  3
HForm:  -0.04264196472992676 EHull:  0.5603974725184155 Classification:  2 Ours:  3
HForm:  -1.2641947301245828 EHull:  0.33341639143359547 Classification:  2 Ours:  3
HForm:  -0.34231478322778397 EHull:  0.3198415093835404 Classification:  2 Ours:  3
HForm:  -0.22133001001785324 EHull:  0.4197104693285789 Classification:  2 Ours:  3
HForm:  0.25098345695080315 EHull:  0.25098345695080315 Classification:  1 Ours:  1
HForm:  -0.08735625780812262 EHull:  0.13863128023547278 Classification:  3

HForm:  -1.1942555651757778 EHull:  0.00099216468469554 Classification:  3 Ours:  3
HForm:  -1.6812846999771969 EHull:  0.14207954587153182 Classification:  3 Ours:  3
HForm:  -1.07795861105411 EHull:  0.05792024730634049 Classification:  3 Ours:  3
HForm:  -0.2297251645104339 EHull:  0.0040945138448381935 Classification:  3 Ours:  3
HForm:  -1.3405545451053107 EHull:  -4.0102055512969415e-05 Classification:  3 Ours:  3
HForm:  0.3147704667815301 EHull:  0.35678702417997066 Classification:  1 Ours:  1
HForm:  -2.582677908811611 EHull:  0.4980051368569014 Classification:  2 Ours:  3
HForm:  -0.9711300925349278 EHull:  0.05385260975214656 Classification:  3 Ours:  3
HForm:  -0.2697148879055035 EHull:  -0.040872165027804996 Classification:  3 Ours:  3
HForm:  -1.572378151378698 EHull:  -0.2670141443097709 Classification:  3 Ours:  3
HForm:  -2.137011283046266 EHull:  0.07669191609923587 Classification:  3 Ours:  3
HForm:  -0.38321899305118085 EHull:  0.3087463310656395 Classification:  2 

HForm:  -0.6127948218876155 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -1.6731815990413506 EHull:  0.03188364119370091 Classification:  3 Ours:  3
HForm:  -1.275809700083002 EHull:  0.30968177641445505 Classification:  2 Ours:  3
HForm:  -0.45490861535336935 EHull:  0.18849886055937548 Classification:  3 Ours:  3
HForm:  -0.790247374089678 EHull:  0.38033752589848113 Classification:  2 Ours:  3
HForm:  -0.7948907767743464 EHull:  0.592218119590593 Classification:  2 Ours:  3
HForm:  -1.7923013848304579 EHull:  0.7622103792753583 Classification:  2 Ours:  3
HForm:  -0.5626765970442932 EHull:  0.8150801225556475 Classification:  2 Ours:  3
HForm:  -1.767788679225042 EHull:  0.009127442701000632 Classification:  3 Ours:  3
HForm:  -2.583980944978562 EHull:  0.20283085983798932 Classification:  2 Ours:  3
HForm:  -1.436608635920288 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.45551276040520533 EHull:  0.166649655680094 Classification:  3 Ours:  3
HForm:  -0.29727478885063546 EHu

In [32]:
db = connect('c2db.db')
length = 0
adapter = AseAtomsAdaptor()
arrayOfThings2 = []
for row in db.select():
    try:
        a = row['hform']
        a = row['ehull']
        varA = row['hform']
        varB = row['ehull']
        tempThis = classificationThermo(float(varA), float(varB))
        if(row['thermodynamic_stability_level'] + 1 != tempThis):
            print("HForm: ", varA, "EHull: ", varB, "Classification: ", (row['thermodynamic_stability_level']), "Ours: ", tempThis+1)
        arrayOfThings2.append(tempThis)
    except:
        pass

HForm:  0.4247370934348935 EHull:  0.4247370934348935 Classification:  1 Ours:  1
HForm:  -1.064860946472334 EHull:  0.04936244159942782 Classification:  3 Ours:  3
HForm:  -0.7433614716609935 EHull:  0.35030501021094795 Classification:  2 Ours:  3
HForm:  -0.16756754153148634 EHull:  0.36280426507292063 Classification:  2 Ours:  3
HForm:  -0.4958010423209753 EHull:  0.03780502792618501 Classification:  3 Ours:  3
HForm:  -0.31377007784410793 EHull:  0.19105647442319462 Classification:  3 Ours:  3
HForm:  -1.0870792714193929 EHull:  -0.002629469935327533 Classification:  3 Ours:  3
HForm:  -1.8548282225228245 EHull:  -0.0670393496229571 Classification:  3 Ours:  3
HForm:  -2.9275122365498882 EHull:  -0.09373423588387153 Classification:  3 Ours:  3
HForm:  -1.270877318478873 EHull:  0.04125800310794148 Classification:  3 Ours:  3
HForm:  -2.954920238785681 EHull:  0.03838200004081349 Classification:  3 Ours:  3
HForm:  -0.15957109937497727 EHull:  -0.05336913346734673 Classification:  3

HForm:  -1.2579108849022065 EHull:  0.040215869243811486 Classification:  3 Ours:  3
HForm:  -0.19158552712787844 EHull:  0.36136687477880347 Classification:  2 Ours:  3
HForm:  -1.6219137250517093 EHull:  0.10671968024854683 Classification:  3 Ours:  3
HForm:  -1.585491476497457 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -1.6575703400756936 EHull:  0.25372522792029595 Classification:  2 Ours:  3
HForm:  -0.5897667907978125 EHull:  0.1675861279458417 Classification:  3 Ours:  3
HForm:  -2.4970091588280545 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -2.287543746602401 EHull:  0.17188120919671812 Classification:  3 Ours:  3
HForm:  -1.067907262108993 EHull:  0.023655733356264363 Classification:  3 Ours:  3
HForm:  -0.6557589834809207 EHull:  0.032819288303295946 Classification:  3 Ours:  3
HForm:  -1.0046006193331294 EHull:  0.11986881004493677 Classification:  3 Ours:  3
HForm:  -0.9422380525473131 EHull:  0.044237402440444384 Classification:  3 Ours:  3
HForm:  -0.348499245629

HForm:  -0.21654895020602627 EHull:  0.5736332802173418 Classification:  2 Ours:  3
HForm:  -0.24536835143273272 EHull:  0.039974789168580216 Classification:  3 Ours:  3
HForm:  -2.3517584441126482 EHull:  0.6614486355201454 Classification:  2 Ours:  3
HForm:  -1.9547054482592747 EHull:  0.7270984950775201 Classification:  2 Ours:  3
HForm:  0.4445696015319278 EHull:  0.49784700362755974 Classification:  1 Ours:  1
HForm:  -0.34610404769762626 EHull:  0.07499368598771872 Classification:  3 Ours:  3
HForm:  0.24444814088836964 EHull:  0.5464214959494909 Classification:  1 Ours:  1
HForm:  -0.27873984058104195 EHull:  0.04968931321426773 Classification:  3 Ours:  3
HForm:  -0.056399607566345544 EHull:  0.44640914189441405 Classification:  2 Ours:  3
HForm:  -0.14502022754655197 EHull:  0.11891256071551137 Classification:  3 Ours:  3
HForm:  -0.6921898712666077 EHull:  0.13723873159341515 Classification:  3 Ours:  3
HForm:  -1.0264364009199403 EHull:  0.571174720638238 Classification:  2 

HForm:  -0.15417195913010176 EHull:  0.09054203815417519 Classification:  3 Ours:  3
HForm:  -0.5219203454229884 EHull:  0.09490989990580345 Classification:  3 Ours:  3
HForm:  0.20048196592494918 EHull:  0.36935280113616786 Classification:  1 Ours:  1
HForm:  -0.6361771330776976 EHull:  0.1288921168351136 Classification:  3 Ours:  3
HForm:  -2.92726188573583 EHull:  0.2219747692428511 Classification:  2 Ours:  3
HForm:  -1.0473442519019516 EHull:  0.0 Classification:  3 Ours:  3
HForm:  0.15883690214632962 EHull:  0.15883690214632962 Classification:  3 Ours:  3
HForm:  -0.13671815714326102 EHull:  0.003172293971253737 Classification:  3 Ours:  3
HForm:  -2.3293440352572365 EHull:  0.09740963432734873 Classification:  3 Ours:  3
HForm:  -0.49991276924851125 EHull:  0.03759865809388696 Classification:  3 Ours:  3
HForm:  -1.6913516924886693 EHull:  0.26329445680028907 Classification:  2 Ours:  3
HForm:  -2.1233676610411587 EHull:  0.13883727508576182 Classification:  3 Ours:  3
HForm:  

HForm:  -0.942826229268794 EHull:  0.24152037785888159 Classification:  2 Ours:  3
HForm:  -2.3073893431821717 EHull:  0.16755116369806755 Classification:  3 Ours:  3
HForm:  -1.0599337492239358 EHull:  0.14499677409937872 Classification:  3 Ours:  3
HForm:  -0.7750804616219916 EHull:  0.2548479303285224 Classification:  2 Ours:  3
HForm:  -0.8871847088040269 EHull:  0.371267550972031 Classification:  2 Ours:  3
HForm:  -0.4914163239979558 EHull:  0.2549223475130176 Classification:  2 Ours:  3
HForm:  -1.599793993467128 EHull:  0.0619092479286949 Classification:  3 Ours:  3
HForm:  -1.031976645715139 EHull:  0.09031173152165817 Classification:  3 Ours:  3
HForm:  -0.9441659361397775 EHull:  0.23345148308609842 Classification:  2 Ours:  3
HForm:  -1.3937899194027186 EHull:  0.17231856400340217 Classification:  3 Ours:  3
HForm:  -1.4563266788383333 EHull:  0.24479003007106814 Classification:  2 Ours:  3
HForm:  -1.5029118733831837 EHull:  0.49448288487499203 Classification:  2 Ours:  3


...

HForm:  -1.4055165346514678 EHull:  0.11671526469545901 Classification:  3 Ours:  3
HForm:  -0.7172640964333302 EHull:  0.07440541420416857 Classification:  3 Ours:  3
HForm:  -1.3498678471855878 EHull:  0.14712739039457645 Classification:  3 Ours:  3
HForm:  -0.6568328678465374 EHull:  0.019942510494939447 Classification:  3 Ours:  3
HForm:  -0.5556046187942135 EHull:  0.0702870198999207 Classification:  3 Ours:  3
HForm:  -1.0523238118300604 EHull:  0.02729563864463147 Classification:  3 Ours:  3
HForm:  -0.16847940779076018 EHull:  0.2379516308544631 Classification:  2 Ours:  3
HForm:  -0.41631156894946564 EHull:  0.2356346647681815 Classification:  2 Ours:  3
HForm:  -0.14792018696406592 EHull:  0.5518726102124751 Classification:  2 Ours:  3
HForm:  -1.3242785237675971 EHull:  0.10174238412433212 Classification:  3 Ours:  3
HForm:  -0.13818402809157368 EHull:  0.18705538000925528 Classification:  3 Ours:  3
HForm:  -2.2430241998888305 EHull:  0.2888446787716674 Classification:  2 O

HForm:  -0.8086488467366332 EHull:  0.4872779176252776 Classification:  2 Ours:  3
HForm:  -1.1445375234700954 EHull:  0.12193367176835679 Classification:  3 Ours:  3
HForm:  -2.304677670296814 EHull:  0.2675926936232238 Classification:  2 Ours:  3
HForm:  -0.4065925735952914 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.04056652036497255 EHull:  0.245593942392014 Classification:  2 Ours:  3
HForm:  0.3202738080054643 EHull:  0.6580124336824855 Classification:  1 Ours:  1
HForm:  -1.1676544319818536 EHull:  0.010168260186153688 Classification:  3 Ours:  3
HForm:  -0.18784174897376596 EHull:  0.1797082408019219 Classification:  3 Ours:  3
HForm:  -0.0857762184181367 EHull:  0.1682125570059686 Classification:  3 Ours:  3
HForm:  -0.03935942826828054 EHull:  0.30977397275454877 Classification:  2 Ours:  3
HForm:  -0.739226792406202 EHull:  0.3101989215887263 Classification:  2 Ours:  3
HForm:  0.6739791970923156 EHull:  0.7246706091660561 Classification:  1 Ours:  1
HForm:  -0.402631

HForm:  -0.25894986412127974 EHull:  0.009961411810699428 Classification:  3 Ours:  3
HForm:  -0.09109144593369933 EHull:  0.5261504942516382 Classification:  2 Ours:  3
HForm:  0.2086949398397957 EHull:  0.46288754070816074 Classification:  1 Ours:  1
HForm:  -0.11573383046869923 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.22777278065656992 EHull:  0.44855597951060366 Classification:  2 Ours:  3
HForm:  -0.037912259491902134 EHull:  0.41783422222041566 Classification:  2 Ours:  3
HForm:  -0.00045421121726807456 EHull:  0.3044902189792005 Classification:  2 Ours:  3
HForm:  0.2830702670648577 EHull:  0.4995910015890912 Classification:  1 Ours:  1
HForm:  -0.07133979383706279 EHull:  0.576112840932764 Classification:  2 Ours:  3
HForm:  -1.2486591087568977 EHull:  0.574705137091831 Classification:  2 Ours:  3
HForm:  -0.22356323685558444 EHull:  0.2415803782994228 Classification:  2 Ours:  3
HForm:  -0.7090855441372512 EHull:  0.1329279694874539 Classification:  3 Ours:  3
HForm:

HForm:  -0.296350161762577 EHull:  0.016832362264916245 Classification:  3 Ours:  3
HForm:  -1.4511840133894751 EHull:  0.14062835630963866 Classification:  3 Ours:  3
HForm:  -0.8869493880943674 EHull:  0.06588272643265236 Classification:  3 Ours:  3
HForm:  -1.5069581415902118 EHull:  0.1526249447386927 Classification:  3 Ours:  3
HForm:  -0.7753794705236418 EHull:  0.05652843254672002 Classification:  3 Ours:  3
HForm:  -1.332279635074401 EHull:  0.02789854947377135 Classification:  3 Ours:  3
HForm:  -0.22926409866914454 EHull:  0.2585225173396235 Classification:  2 Ours:  3
HForm:  -0.36159346165471956 EHull:  0.09570476757465257 Classification:  3 Ours:  3
HForm:  -0.4737080763868053 EHull:  0.30455329115113045 Classification:  2 Ours:  3
HForm:  -1.5301171089888932 EHull:  0.0 Classification:  3 Ours:  3
HForm:  -0.11912479150587292 EHull:  0.518655085553109 Classification:  2 Ours:  3
HForm:  -0.4248176824307978 EHull:  0.15443399326776114 Classification:  3 Ours:  3
HForm:  -0

HForm:  -0.04127068501621759 EHull:  0.3470440475550415 Classification:  2 Ours:  3
HForm:  0.5133258115374311 EHull:  0.8434192774058502 Classification:  1 Ours:  1
HForm:  0.34823210602719357 EHull:  0.5375059151298415 Classification:  1 Ours:  1
HForm:  0.4709007249917286 EHull:  1.0257479236068594 Classification:  1 Ours:  1
HForm:  0.8939798334741168 EHull:  0.9446712455478573 Classification:  1 Ours:  1
HForm:  -0.08778445354877225 EHull:  0.3080405014997396 Classification:  2 Ours:  3
HForm:  -0.8484425660143788 EHull:  0.6153488547135393 Classification:  2 Ours:  3
HForm:  -0.2866477276207071 EHull:  0.09080674001249517 Classification:  3 Ours:  3
HForm:  -0.3236837971925694 EHull:  0.20591233013779936 Classification:  2 Ours:  3
HForm:  -0.14972497235934887 EHull:  0.40915663917997835 Classification:  2 Ours:  3
HForm:  -0.30227358918598907 EHull:  0.48749071159859864 Classification:  2 Ours:  3
HForm:  -0.8816327669456031 EHull:  0.26377061329281226 Classification:  2 Ours:  

In [62]:
print(len(structs))
print(length)
print(len(elasticityClassifications))

13896
4047
2095


In [33]:
print(arrayOfThings)

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 

In [64]:
tqdm.pandas()
df = pd.DataFrame()
totalNames = []
for i in range(len(structs)):
    totalNames.append(i)
df['material_id'] = totalNames[:6063]
df['final_structure'] = structs[:6063]
df['dynamic_stability_phonons'] = elasticityClassifications[:6063]

In [34]:
print(arrayOfThings2)

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 

In [69]:
structs[3]

Structure Summary
Lattice
    abc : 6.75373502001685 6.75373502001685 6.7537348264903025
 angles : 113.3808195729302 113.3808195729302 113.38082800859058
 volume : 195.45560951606652
      A : 3.708899 -5.644201 0.0
      B : 3.708899 5.644201 0.0
      C : -4.880442 0.0 4.668428
PeriodicSite: Se (-1.1095, 3.5333, 4.0263) [0.1049, 0.7309, 0.8624]
PeriodicSite: Se (5.3977, 0.7427, 0.4895) [0.7309, 0.8624, 0.1049]
PeriodicSite: Se (2.5167, 4.2760, 1.2565) [0.1376, 0.8951, 0.2691]
PeriodicSite: Se (3.6469, -3.5333, 0.6422) [0.8951, 0.2691, 0.1376]
PeriodicSite: Se (0.0207, -4.2760, 3.4120) [0.8624, 0.1049, 0.7309]
PeriodicSite: Se (-2.8603, -0.7427, 4.1789) [0.2691, 0.1376, 0.8951]

In [65]:
print(df)

      material_id                                    final_structure  \
0               0  [[0.32130853 0.32130853 0.32130853] N, [2.6066...   
1               1                                    [[0. 0. 0.] Tl]   
2               2                                    [[0. 0. 0.] Ne]   
3               3  [[-1.10953652  3.53331498  4.02625772] Se, [5....   
4               4   [[1.367182 1.367182 1.367182] Si, [0. 0. 0.] Si]   
...           ...                                                ...   
6058         6058  [[ 4.106073   -2.37064707  3.46797281] Rb, [0....   
6059         6059  [[ 5.49641398  9.18706607 10.0757325 ] Er, [0....   
6060         6060  [[3.8739756  2.58834818 2.86241866] Pb, [3.127...   
6061         6061  [[6.40251324 0.         5.545335  ] Ba, [6.402...   
6062         6062  [[4.80942747 3.60283814 2.81154572] La, [ 4.83...   

      dynamic_stability_phonons  
0                             1  
1                             0  
2                             1  

In [74]:
print(materialNamesStiffness[6000], "\n", structs[6000], "\n", elasticityClassifications[6000])

a9da27c1918419d649ce4b813b06aba2 
 Full Formula (K2 Ga2 P8 O24)
Reduced Formula: KGa(PO3)4
abc   :  10.526581  10.526581   5.221018
angles:  81.252495  81.252495  72.356213
Sites (36)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  K     0.407052  0.407052  0.739738        -0
  1  K     0.592948  0.592948  0.260262        -0
  2  Ga    0.5       0         0               -0
  3  Ga    0         0.5       0               -0
  4  P     0.799344  0.354992  0.83615         -0
  5  P     0.0686    0.308842  0.533534         0
  6  P     0.308842  0.0686    0.533534        -0
  7  P     0.645008  0.200656  0.16385         -0
  8  P     0.354992  0.799344  0.83615         -0
  9  P     0.691158  0.9314    0.466466        -0
 10  P     0.9314    0.691158  0.466466        -0
 11  P     0.200656  0.645008  0.16385         -0
 12  O     0.426797  0.849401  0.014198         0
 13  O     0.399297  0.10039   0.698601         0
 14  O     0.430

In [75]:
df['wyckoff'] = df['final_structure'].progress_apply(get_aflow_label_spglib)
df['composition'] = df['final_structure'].apply(lambda x: x.composition.reduced_formula)

  0%|          | 0/6063 [00:00<?, ?it/s]C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
 59%|█████▉    | 3580/6063 [00:27<00:23, 104.19it/s]C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
100%|██████████| 6063/6063 [00:42<00:00, 141.81it/s]
C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\dylan\anaconda3\envs\roost\lib\site-pac

In [76]:
df

,material_id,final_structure,dynamic_stability_phonons,wyckoff,composition
0,0,"[[0.32130853 0.32130853 0.32130853] N, [2.6066...",1,A_cP8_205_c:N,N2
1,1,[[0. 0. 0.] Tl],0,A_cI2_229_a:Tl,Tl
2,2,[[0. 0. 0.] Ne],1,A_cF4_225_a:Ne,Ne
3,3,"[[-1.10953652 3.53331498 4.02625772] Se, [5....",0,A_hR18_148_f:Se,Se
4,4,"[[1.367182 1.367182 1.367182] Si, [0. 0. 0.] Si]",1,A_cF8_227_a:Si,Si
...,...,...,...,...,...
6058,6058,"[[ 4.106073 -2.37064707 3.46797281] Rb, [0....",1,A3B10C_hP56_186_2c_2b4cd_ab:Er-F-Rb,RbEr3F10
6059,6059,"[[ 5.49641398 9.18706607 10.0757325 ] Er, [0....",1,A3BC10D2E_oP68_52_ce_d_5e_e_c:Er-F-O-Se-Si,Er3SiSe2O10F
6060,6060,"[[3.8739756 2.58834818 2.86241866] Pb, [3.127...",1,AB2C3D2_oP32_60_c_d_cd_d:C-F-O-Pb,Pb2CO3F2
6061,6061,"[[6.40251324 0. 5.545335 ] Ba, [6.402...",1,ABC4D_oC28_40_a_b_2bc_b:Ba-Hg-S-Sn,BaSnHgS4


In [77]:
df = df.sort_values(by=['wyckoff'], ascending=True)
df_wyk = df.drop_duplicates(['wyckoff'], keep='first')

In [78]:
df_wyk = df_wyk.sort_values(by=['composition'], ascending=True)
df_comp = df_wyk.drop_duplicates('composition', keep='first')

In [79]:
fields = ['material_id', 'composition', 'wyckoff', 'dynamic_stability_phonons']

In [81]:
df_wyk[fields].to_csv('C:\\Users\\dylan\\Desktop\\generatedData\\stiffnessStabilityv2.csv', index=False)

In [11]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from ase.db import connect
from scipy.sparse import identity
from scipy.sparse.linalg import eigs

# Connect to database
db = connect('c2db.db')

properties = ['hform', 'ehull', 'minhessianeig', 'c_11','c_12','c_13','c_21', 'c_22','c_23','c_31','c_32','c_33']
rows = [[] for value in range(len(properties))]
equation = [[] for value in range(len(properties))]
values = [[] for value in range(len(properties))]
for index in range(len(properties)):
    rows[index] = db.select(properties[index])
    for entry in rows[index]:
        equation[index].append(entry['formula'])
        values[index].append(entry[properties[index]])
for index in range(len(properties)):
    print(properties[index], " Entries: ", len(equation[index]))
intersectionResult = list(set(equation[0]).intersection(*equation))
print("Feasible Combined Datapoints:", len(intersectionResult))
equationsTogether = intersectionResult
propertiesTogether = [[] for value in range(len(properties))]
for entry in equationsTogether:
    for index in range(len(propertiesTogether)):
        itemIndex = equation[index].index(entry)
        propertiesTogether[index].append(values[index][itemIndex])


hform  Entries:  4044
ehull  Entries:  4044
minhessianeig  Entries:  3859
c_11  Entries:  3968
c_12  Entries:  3968
c_13  Entries:  3968
c_21  Entries:  3968
c_22  Entries:  3968
c_23  Entries:  3968
c_31  Entries:  3968
c_32  Entries:  3968
c_33  Entries:  3968
Feasible Combined Datapoints: 2656


In [13]:
#Not necessary
for propertyIndex in range(len(propertiesTogether)):
    for elementIndex in range(len(propertiesTogether[0])):
        element = propertiesTogether[propertyIndex][elementIndex]
        if(element == 'high'):
            propertiesTogether[propertyIndex][elementIndex] = 1
        if(element == 'low'):
            propertiesTogether[propertyIndex][elementIndex] = 0
minEigenvaluesFinal = []
for index in range(len(propertiesTogether[0])):
    valuesTmp = []
    for propertyIndex in range(3, len(properties)):
        valuesTmp.append(propertiesTogether[propertyIndex][index])
    minEigenvalueHere = findMinEigenvalue(valuesTmp[0],valuesTmp[1],valuesTmp[2],valuesTmp[3],valuesTmp[4],valuesTmp[5],valuesTmp[6],valuesTmp[7],valuesTmp[8])
    minEigenvaluesFinal.append(minEigenvalueHere)
    
propertiesTogether = propertiesTogether[0:len(propertiesTogether)-9]
print(len(propertiesTogether))
propertiesTogether.append(minEigenvaluesFinal)

IndexError: list index out of range

In [ ]:
import csv
index = 0
databaseLocation = "C:\\Users\\dylan\\Desktop\\generatedData\\"
print(databaseLocation)
properties = ['hform', 'ehull', 'minhessianeig', 'minstiffeig']

with open(str(databaseLocation + 'dataCombinedRegression.csv'),'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    arrayOfEntries = ['material_id','composition']
    for propertyEntry in properties:
        arrayOfEntries.append(propertyEntry)
    csvwriter.writerow(arrayOfEntries)
    for equationTitle in equationsTogether:
        array = []
        array.append(index)
        array.append(equationsTogether[index])
        for propertyIndex in range(len(propertiesTogether)):
            array.append(propertiesTogether[propertyIndex][index])
        csvwriter.writerow(array)
        index = index + 1